In [10]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
from devtools import debug

[sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent/"python"] if str(path) not in sys.path]

[None, None]

In [1]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.messages import HumanMessage

chat = ChatLiteLLM(model="gpt-3.5-turbo")

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat(messages)

ImportError: cannot import name 'ChatLiteLLM' from 'langchain_community.chat_models' (/home/tcl/.cache/pypoetry/virtualenvs/genai-training-GvhVNXsu-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/__init__.py)

In [5]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("user", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("user", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


In [13]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

from langchain_experimental.chat_models import Llama2Chat

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)
messages = chat_prompt.format_messages( word_count = 10, conversation = messages)
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?'), HumanMessage(content='Summarize our conversation so far in 10 words.'), HumanMessage(content='Summarize our conversation so far in 10 words.')]


In [39]:
from typing import List, Optional, cast
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.prompts import PromptTemplate

from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatGeneration,
    ChatResult,
    HumanMessage,
    LLMResult,
    SystemMessage,
)

from langchain_core.messages.base import BaseMessage
from pydantic import BaseModel


DEFAULT_SYSTEM_PROMPT = ""

class PromptFormatter(BaseModel):

    system_message: SystemMessage = SystemMessage(content=DEFAULT_SYSTEM_PROMPT)

    text_beg : str = ""
    sys_beg: str
    sys_end: str
    ai_n_beg: str
    ai_n_end: str
    usr_n_beg: str
    usr_n_end: str
    usr_0_beg: Optional[str] = None
    usr_0_end: Optional[str] = None
    text_end : str =""

    def _to_chat_prompt(
        self,
        messages: List[BaseMessage],
    ) -> str:
        """Convert a list of messages into a prompt format expected by wrapped LLM."""
        if not messages:
            raise ValueError("at least one HumanMessage must be provided")

        if not isinstance(messages[0], SystemMessage):
            messages = [self.system_message] + messages

        if not isinstance(messages[1], HumanMessage):
            raise ValueError(
                "messages list must start with a SystemMessage or UserMessage"
            )

        if not isinstance(messages[-1], HumanMessage):
            raise ValueError("last message must be a HumanMessage")

        prompt_parts = [self.text_beg]

        if self.usr_0_beg is None:
            self.usr_0_beg = self.usr_n_beg

        if self.usr_0_end is None:
            self.usr_0_end = self.usr_n_end

        prompt_parts.append(
            self.sys_beg + cast(str, messages[0].content) + self.sys_end
        )
        prompt_parts.append(
            self.usr_0_beg + cast(str, messages[1].content) + self.usr_0_end
        )

        for ai_message, human_message in zip(messages[2::2], messages[3::2]):
            prompt_parts.append(
                self.ai_n_beg + cast(str, ai_message.content) + self.ai_n_end
            )
            prompt_parts.append(
                self.usr_n_beg + cast(str, human_message.content) + self.usr_n_end
            )

        prompt_parts.append(self.text_end)
        return "".join(prompt_parts)


class Llama2Format(PromptFormatter):
    """Wrapper for Llama-2-chat model."""
    # 

    sys_beg: str = "<s>[INST] <<SYS>>\n"
    sys_end: str = "\n<</SYS>>\n\n"
    ai_n_beg: str = " "
    ai_n_end: str = " </s>"
    usr_n_beg: str = "<s>[INST] "
    usr_n_end: str = " [/INST]"
    usr_0_beg: str = ""
    usr_0_end: str = " [/INST]"

class Llama3Format(PromptFormatter):
    """Wrapper for Llama-2-chat model."""

    text_beg: str = "<|begin_of_text|>"
    sys_beg: str = "<|start_header_id|>system<|end_header_id|>"
    sys_end: str = "<|eot_id|>\n"
    ai_n_beg: str = "<|start_header_id|>assistant<|end_header_id|>"
    ai_n_end: str = "<|eot_id|>"
    usr_n_beg: str = "<|start_header_id|>user<|end_header_id|>"
    usr_n_end: str = "<|eot_id|>"
    text_end: str = "<|start_header_id|>assistant<|end_header_id|>"


class MixtralFormat(PromptFormatter):
    """See https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1#instruction-format"""  # noqa: E501

    # https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-chat

    @property
    def _llm_type(self) -> str:
        return "mixtral"

    sys_beg: str = "<s>[INST] "
    sys_end: str = "\n"
    ai_n_beg: str = " "
    ai_n_end: str = " </s>"
    usr_n_beg: str = " [INST] "
    usr_n_end: str = " [/INST]"
    usr_0_beg: str = ""
    usr_0_end: str = " [/INST]"



In [37]:

formatter = Llama3Format()
formatter._to_chat_prompt(messages)


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>system message<|eot_id|><|start_header_id|>user<|end_header_id|>user message<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [38]:
prompt_original = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> system message <|eot_id|>
    <|start_header_id|>user<|end_header_id|>user message <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=[],
)
prompt_original.format()

' <|begin_of_text|><|start_header_id|>system<|end_header_id|> system message <|eot_id|>\n    <|start_header_id|>user<|end_header_id|>user message <|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [35]:
new_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "system message"),
        ("user", "user message"),
    ]
)
messages = new_prompt.format_messages()
formatter._to_chat_prompt(messages)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>system message<|eot_id|><|start_header_id|>user<|end_header_id|>user message<|eot_id|>'

In [47]:
from textwrap import dedent
from typing import List, Optional, cast
from langchain_core.prompts import ChatPromptTemplate, BasePromptTemplate
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

from langchain.schema import (
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_core.messages.base import BaseMessage
from pydantic import BaseModel
from devtools import debug



def to_chat_prompt(
    input_prompt: ChatPromptTemplate,
) :
    
    """Convert a list of messages into a prompt format expected by wrapped LLM."""


    template = []
    input_variables = []

    debug(input_prompt)
    for message in input_prompt.messages:
        if isinstance(message, SystemMessagePromptTemplate):
            template.append(
                "self.sys_beg" + dedent(cast(str, message.prompt.template) + "self.sys_end"
            ))
        elif 


new_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """this is a system message talking about {topic}"""),
        (
            "user",
            """
            This is a long
            user message""",
        ),
    ]
)

to_chat_prompt(new_prompt)

/tmp/ipykernel_322598/1230483511.py:30 to_chat_prompt
    input_prompt: ChatPromptTemplate(
        input_variables=['topic'],
        messages=[
            SystemMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    template='this is a system message talking about {topic}',
                ),
            ),
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=[],
                    template=(
                        '\n'
                        '            This is a long\n'
                        '            user message'
                    ),
                ),
            ),
        ],
    ) (ChatPromptTemplate) len=2
/tmp/ipykernel_322598/1230483511.py:32 to_chat_prompt
    prompt.input_variables: [
        'documents',
        'generation',
    ] (list) len=2
/tmp/ipykernel_322598/1230483511.py:32 to_chat_prompt
    prompt.input_variables: 